In [2]:
!pip install segmentation_models_pytorch
!pip install wandb
!pip install torcheval
!pip install torchmetrics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.7/106.7 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 6.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 13.9 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_6

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Iniatial

In [ ]:
!mkdir Dataset
!mkdir Dataset/IMD2020

!wget -P Dataset/IMD2020 https://staff.utia.cas.cz/novozada/db/IMD2020.zip

mkdir: cannot create directory ‘Dataset’: File exists
mkdir: cannot create directory ‘Dataset/IMD2020’: File exists
--2024-05-15 16:32:05--  https://staff.utia.cas.cz/novozada/db/IMD2020.zip
Resolving staff.utia.cas.cz (staff.utia.cas.cz)... 147.231.16.197
Connecting to staff.utia.cas.cz (staff.utia.cas.cz)|147.231.16.197|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 592836398 (565M) [application/zip]
Saving to: ‘Dataset/IMD2020/IMD2020.zip’

IMD2020.zip           0%[                    ]   8.00K  38.0KB/s               ^C


In [6]:
!mkdir Dataset
!mkdir Dataset/IMD2020

!unzip /content/drive/MyDrive/IMD2020.zip -d /content/Dataset/IMD2020

Archive:  /content/drive/MyDrive/IMD2020.zip
   creating: /content/Dataset/IMD2020/1a1ogs/
  inflating: /content/Dataset/IMD2020/1a1ogs/1a1ogs_orig.jpg  
  inflating: /content/Dataset/IMD2020/1a1ogs/c8tf5mq_0.png  
  inflating: /content/Dataset/IMD2020/1a1ogs/c8tf5mq_0_mask.png  
   creating: /content/Dataset/IMD2020/1a3oag/
  inflating: /content/Dataset/IMD2020/1a3oag/1a3oag_orig.jpg  
  inflating: /content/Dataset/IMD2020/1a3oag/c8tt7fg_0.jpg  
  inflating: /content/Dataset/IMD2020/1a3oag/c8tt7fg_0_mask.png  
  inflating: /content/Dataset/IMD2020/1a3oag/c8ttni9_0.jpg  
  inflating: /content/Dataset/IMD2020/1a3oag/c8ttni9_0_mask.png  
  inflating: /content/Dataset/IMD2020/1a3oag/c8ttxcu_0.jpg  
  inflating: /content/Dataset/IMD2020/1a3oag/c8ttxcu_0_mask.png  
  inflating: /content/Dataset/IMD2020/1a3oag/c8ttxi0_0.jpg  
  inflating: /content/Dataset/IMD2020/1a3oag/c8ttxi0_0_mask.png  
  inflating: /content/Dataset/IMD2020/1a3oag/c8tvsnn_0.jpg  
  inflating: /content/Dataset/IMD2020/1a3

In [4]:
!rm -r /content/drive/MyDrive/IMD2020.zip

In [ ]:
# import shutil

# shutil.rmtree('/Project/Dataset', ignore_errors=True)

# Libraries

In [1]:
# For managing COCO dataset
# from pycocotools.coco import COCO

# For creating and managing folder/ files
import glob
import os
import shutil

# For managing images
from PIL import Image
import skimage.io as io

# Basic libraries
import numpy as np
import pandas as pd
import random
import cv2

# For plotting
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import seaborn as sns
import wandb

# For importing models and working with them
## Torch
import torch
import torch.utils.data # for Dataset
import torch.nn as nn
from torch.optim import Adam
from torch.autograd import Variable
from torch.utils.data import DataLoader, Dataset
import segmentation_models_pytorch as smp

## Torchvision
import torchvision
from torchvision.transforms import transforms

# For creating train - test splits
from sklearn.model_selection import train_test_split

import pathlib
import pylab
import requests
from io import BytesIO
from pprint import pprint
from tqdm import tqdm
import time
from imutils import paths

# Performance Metrics
from sklearn.metrics import multilabel_confusion_matrix
from sklearn.metrics import confusion_matrix


# Functions - to have them separated in files
import CalcMetrics as cm
import BinaryMasks as bm
import TrainModel as trModel
import ValidateModel as valModel
import TestModel as testModel
import CreateDataset_Comofod as com
import CreateDataset_IMD2020 as imd
import DisplayMetrics as dm
import PlotResults as pr
import MainLoop as main
import EarlyStopping as stopping


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)


# %matplotlib inline

cuda
cuda
cuda
cuda
cuda
cuda
cuda
cuda
cuda
cuda
cuda
cuda
cuda
cuda


# Data

## Data Preparation

In [2]:
dataset_path = 'Dataset/IMD2020'
num_folders = 400  # Number of folders to read

original_images, altered_images, masks = imd.load_dataset(dataset_path, num_folders)


Starting loading dataset
Error loading images for folder 1d6mhe: [Errno 2] No such file or directory: 'Dataset/IMD2020/1d6mhe/1d6mhe_orig.jpg'
No altered image found for folder z10
No altered image found for folder z11
No altered image found for folder z12
No altered image found for folder z13
No altered image found for folder z14
No altered image found for folder z15
No altered image found for folder z16
No altered image found for folder z17
No altered image found for folder z18
No altered image found for folder z19
No altered image found for folder z2
No altered image found for folder z20
No altered image found for folder z21
No altered image found for folder z22
No altered image found for folder z23
No altered image found for folder z24
No altered image found for folder z25
No altered image found for folder z26
No altered image found for folder z27
No altered image found for folder z28
No altered image found for folder z29
No altered image found for folder z3
No altered image found 

In [3]:
len(original_images)

373

In [4]:
len(altered_images)

373

In [5]:
# 73c482950d37419655b401f882c34c719ca587af

## Main

In [ ]:
RESNET_SIZE = 256
transf_size = RESNET_SIZE

# Initialize transformations - train
transforms_train = transforms.Compose([
    transforms.Resize((transf_size, transf_size)),
    transforms.ToTensor(),
])

# Initialize transformations - test (optional)
transforms_test = transforms.Compose([
    transforms.Resize((transf_size, transf_size)),
    transforms.ToTensor(),
])

# Initialize the U-Net model
# unet = smp.Unet(
#                 encoder_name = "resnet101",
#                 encoder_weights = "imagenet",
#                 in_channels = 3,  # 3 channels for the image
#                 classes = 1,  # 1 class => binary mask
#                 activation = 'sigmoid'
#                ).to(device)


# Define hyperparameters
INIT_LR = 0.0001
BATCH_SIZE = 8
NUM_EPOCHS = [10, 20, 50, 100, 150]
TEST_SPLIT = [0.1, 0.2] # 0.3
VALID_SPLIT = 0.2 # 0.1

model_type = 'GCA' # unet
channels = 3 # 6
dataset_type = 'imd' # imd, comofod

wb_name = "GCANet_IMD2020_altered_imgs"

main.main_loop(original_images, altered_images, masks, transforms_train, transforms_test,
               model_type, channels, dataset_type,
               wb_name, INIT_LR, BATCH_SIZE, NUM_EPOCHS, TEST_SPLIT, VALID_SPLIT)





wandb: Currently logged in as: bianca-apostolescu (bianca-team). Use `wandb login --relogin` to force relogin


[INFO] TEST_SPLIT = 0.1 ...
Splits, Datasets, and Dataloaders
[INFO] Total time taken to create the dataset and dataloader: 0.00s
trainSteps = 33, testSteps = 4, valSteps = 8


[INFO] Training the network for 10 epochs...


  0%|          | 0/10 [00:00<?, ?it/s]

Training...


...

Training...
Validating...


 99%|█████████▉| 149/150 [44:27<00:17, 17.83s/it]

Training...
Validating...


100%|██████████| 150/150 [44:45<00:00, 17.91s/it]


[INFO] Total time taken to train and validate the model: 2686.00s
Testing...






















TypeError: plot_results() takes 1 positional argument but 5 were given